# SAMSum Heterogeneous 18-Model Swarm (Ollama) — Colab Notebook

This notebook runs a heterogeneous swarm of **18 Ollama models** on the **SAMSum** dataset and saves per-sample outputs to Google Drive as JSONL.

**Consensus method:** SentenceTransformer embeddings + cosine similarity (select the most "central" summary).

**Notes**
- Concurrency is capped to keep Colab stable.
- Put `samsum.jsonl` in `/content/` (Colab file area).


In [ ]:
!pip -q install rouge-score bert-score sentence-transformers scikit-learn spacy pandas tqdm httpx fastapi uvicorn
!python -m spacy download en_core_web_sm -q

# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh
!ollama --version


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
from google.colab import drive
import os, time, json, asyncio, subprocess
from pathlib import Path

BASE_DIR = "/content/slm_evaluation"
RESULTS_DIR = f"{BASE_DIR}/results"
os.makedirs(RESULTS_DIR, exist_ok=True)

DATASET_PATH = "/content/samsum.jsonl"  # upload file to /content/
print("RESULTS_DIR:", RESULTS_DIR)
print("DATASET_PATH:", DATASET_PATH)


Mounted at /content/drive
RESULTS_DIR: /content/drive/MyDrive/slm_evaluation/results
DATASET_PATH: /content/samsum.jsonl


In [61]:
# Persist models locally (ephemeral, as requested)
import os, subprocess, time

# Model storage (your choice, unchanged)
os.environ["OLLAMA_MODELS"] = "/content/"
os.makedirs(os.environ["OLLAMA_MODELS"], exist_ok=True)

os.environ["OLLAMA_NUM_PARALLEL"] = "8"          # increase internal parallelism
os.environ["OLLAMA_MAX_LOADED_MODELS"] = "18"    # keep all models resident
os.environ["OLLAMA_KEEP_ALIVE"] = "24h"          # prevent model eviction

# Stop stale server if any
!pkill -f "ollama serve" || true

# Start and log
!nohup ollama serve > /tmp/ollama.log 2>&1 &

# Wait for readiness
import requests
for i in range(60):
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=2)
        if r.status_code == 200:
            print("OK: Ollama is up")
            break
    except Exception:
        time.sleep(1)
else:
    print("ERROR: Ollama did not start. Showing last log lines:")
    !tail -n 120 /tmp/ollama.log

# Show current tags
!curl -s http://127.0.0.1:11434/api/tags | head -c 2000


^C
OK: Ollama is up
{"models":[{"name":"stablelm2:1.6b","model":"stablelm2:1.6b","modified_at":"2026-01-03T00:39:19.388393534Z","size":982790462,"digest":"714a6116cffa8b415b52c62a7a2d09ba6227ed733baa0025c937a36aee5504f3","details":{"parent_model":"","format":"gguf","family":"stablelm","families":["stablelm"],"parameter_size":"2B","quantization_level":"Q4_0"}},{"name":"tinyllama:1.1b","model":"tinyllama:1.1b","modified_at":"2026-01-03T00:39:07.826291773Z","size":637700138,"digest":"2644915ede352ea7bdfaff0bfac0be74c719d5d5202acb63a6fb095b52f394a4","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"1B","quantization_level":"Q4_0"}},{"name":"orca-mini:7b","model":"orca-mini:7b","modified_at":"2026-01-03T00:38:57.525310185Z","size":3825807733,"digest":"9c9618e2e8952fa2c2fac4986cc128923bd36f239639d7b31ad2692f26de6dab","details":{"parent_model":"","format":"gguf","family":"llama","families":null,"parameter_size":"7B","quantization_level":"Q4_0

In [25]:
!ollama list

NAME    ID    SIZE    MODIFIED 


In [26]:
MODELS = [
  "qwen2.5:7b-instruct",
  "llama3.1:8b",
  "mistral:7b-instruct",
  "phi3:mini",
  "phi3:medium",
  "gemma:7b-instruct",
  "gemma:2b",
  "qwen2.5:3b-instruct",
  "qwen2.5:1.5b-instruct",
  "llama3.2:3b",
  "llama3.2:1b",
  "openchat:7b",
  "neural-chat:7b",
  "dolphin-mistral:7b",
  "zephyr:7b",
  "orca-mini:7b",
  "tinyllama:1.1b",
  "stablelm2:1.6b"
]
print('Model count:', len(MODELS))

Model count: 18


In [27]:
# Pull all models (this can take a while; reruns will be fast if cached in Drive)
assert len(MODELS) == 18, f"Expected 18 models, got {len(MODELS)}"

for m in MODELS:
    print("Pulling:", m)
    !ollama pull $m

# Verify
!curl -s http://127.0.0.1:11434/api/tags | head -c 4000


Pulling: qwen2.5:7b-instruct

Pulling: llama3.1:8b

Pulling: mistral:7b-instruct

Pulling: phi3:mini

Pulling: phi3:medium

Pulling: gemma:7b-instruct

Pulling: gemma:2b

Pulling: qwen2.5:3b-instruct

Pulling: qwen2.5:1.5b-instruct

Pulling: llama3.2:3b

Pulling: llama3.2:1b

Pulling: openchat:7b

Pulling: neural-chat:7b

Pulling: dolphin-mistral:7b

Pulling: zephyr:7b

Pulling: orca-mini:7b

Pulling: tinyllama:1.1b

Pulling: stablelm2:1.6b

{"models":[{"name":"stablelm2:1.6b","model":"stablelm2:1.6b","modified_at":"2026-01-03T00:39:19.388393534Z","size":982790462,"digest":"714a6116cffa8b415b52c62a7a2d09ba6227ed733baa0025c937a36aee5504f3","details":{"parent_model":"","format":"gguf","family":"stablelm","families":["stablelm"],"parameter_size":"2B","quantization_level":"Q4_0"}},{"name":"tinyllama:1.1b","model":"tinyllama:1.1b","modified_at":"2026-01-03T00:39:07.826291773Z","size":637700138,"digest":"2644915ede352ea7bdfaff0bfac0be74c719d5d5202acb63a6fb095b52f394a4","details":{"parent_mod

In [66]:
import os, time, json, asyncio
from pathlib import Path

# pick options (define if not already defined)
try:
    RUN_OPTIONS
except NameError:
    RUN_OPTIONS = {"temperature": 0.2, "num_predict": 256, "num_ctx": 4096}

def _truncate(s: str, n: int = 400):
    if s is None:
        return ""
    s = str(s)
    return s if len(s) <= n else s[:n] + " ...[truncated]"

def _format_candidate_block(c, idx=None):
    header = f"MODEL {idx+1:02d}: {c['model']}" if idx is not None else f"MODEL: {c['model']}"
    lat = f"{c.get('latency', 0.0):.2f}s"
    att = c.get("attempts", 1)
    status = "OK" if not str(c.get("summary","")).startswith("ERROR") else "ERROR"
    return (
        f"{header}\n"
        f"  status: {status} | latency: {lat} | attempts: {att}\n"
        f"  summary: {c.get('summary','')}\n"
    )

async def call_with_retry(model, prompt, client, options, retries=3):
    last_txt, last_lat = None, None
    for k in range(1, retries + 1):
        txt, lat = await call_ollama(model, prompt, client, options)  # <-- FIX: pass options
        if isinstance(txt, str) and not txt.startswith("ERROR"):
            return txt, lat, k
        last_txt, last_lat = txt, lat
        await asyncio.sleep(0.5 * k)
    return last_txt, last_lat, retries

async def run_eval_hetero18(
    max_samples=None,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    write_text_log=True,
    options=RUN_OPTIONS,   # <-- FIX: provide options here
):
    if not os.path.exists(DATASET_PATH):
        print(f"ERROR: Dataset not found at {DATASET_PATH}. Upload samsum.jsonl to /content/")
        return

    if not await ollama_is_up():
        print("ERROR: Ollama not reachable at http://127.0.0.1:11434")
        print("Try: !tail -n 120 /tmp/ollama.log")
        return

    dataset = []
    with open(DATASET_PATH, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(json.loads(line))

    if max_samples is not None:
        dataset = dataset[:max_samples]

    run_id = f"colab_hetero18_{int(time.time())}"
    run_dir = Path(RESULTS_DIR) / run_id
    run_dir.mkdir(parents=True, exist_ok=True)
    out_file = run_dir / "outputs.jsonl"
    log_file = run_dir / "run_log.txt"

    print(f"Starting Evaluation: {run_id}")
    print(f"Results saved to: {out_file}")
    if write_text_log:
        print(f"Text log saved to: {log_file}")
    print(f"Models: {len(MODELS)} | concurrency={concurrency} | samples={len(dataset)}")
    print("-" * 80)

    sem = asyncio.Semaphore(concurrency)

    def _log_line(s: str):
        if write_text_log:
            with open(log_file, "a", encoding="utf-8") as lf:
                lf.write(s + "\n")

    async with httpx.AsyncClient(timeout=300.0) as client:
        for idx_item, item in enumerate(tqdm(dataset, desc="Processing Samples")):
            prompt = f"Summarize this message in one short sentence:\n\n{item['message']}\n\nSummary:"

            async def _one(m):
                async with sem:
                    txt, lat, attempts = await call_with_retry(m, prompt, client, options, retries=3)
                    return m, txt, lat, attempts

            results = await asyncio.gather(*[_one(m) for m in MODELS])

            candidates = []
            for (m, txt, lat, attempts) in results:
                candidates.append({
                    "model": m,
                    "summary": txt,
                    "latency": float(lat),
                    "attempts": int(attempts),
                })

            valid_summaries = [
                c["summary"] for c in candidates
                if isinstance(c["summary"], str) and not c["summary"].startswith("ERROR")
            ]

            if valid_summaries:
                best_idx_valid, meta = get_consensus(valid_summaries)
                swarm_summary = valid_summaries[best_idx_valid]
            else:
                meta = {}
                swarm_summary = "ERROR: All models failed"

            do_print = print_each and ((idx_item % print_every) == 0)
            if do_print:
                sep = "=" * 90
                header = f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}"
                msg_preview = _truncate(item["message"].replace("\n", " "), 400)

                print("\n" + sep)
                print(header)
                print("MESSAGE (preview):", msg_preview)
                print("-" * 90)

                _log_line("\n" + sep)
                _log_line(header)
                _log_line("MESSAGE:")
                _log_line(item["message"])
                _log_line("-" * 90)

                for j, c in enumerate(candidates):
                    c_out = dict(c)
                    if truncate_console and isinstance(c_out["summary"], str):
                        c_out["summary"] = _truncate(c_out["summary"], truncate_console)
                    block = _format_candidate_block(c_out, idx=j)
                    print(block)
                    _log_line(_format_candidate_block(c, idx=j))

                print("-" * 90)
                print("CONSENSUS avg_similarity:", meta.get("avg_similarity"))
                if meta.get("consensus_scores") is not None:
                    print("CONSENSUS scores:", meta.get("consensus_scores"))
                print("SWARM SUMMARY:", swarm_summary)
                print(sep + "\n")

                _log_line("-" * 90)
                _log_line(f"CONSENSUS avg_similarity: {meta.get('avg_similarity')}")
                _log_line(f"CONSENSUS scores: {meta.get('consensus_scores')}")
                _log_line(f"SWARM SUMMARY: {swarm_summary}")
                _log_line(sep)

            result = {
                "id": item.get("id"),
                "swarm_summary": swarm_summary,
                "swarm_candidates": candidates,
                "consensus_metadata": meta,
                "factuality": check_factuality(item["message"], swarm_summary),
            }

            with open(out_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(result) + "\n")

    print(f"\n✓ Done. outputs.jsonl: {out_file}")
    if write_text_log:
        print(f"✓ Done. run_log.txt:  {log_file}")

# Run
await run_eval_hetero18(
    max_samples=None,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    options=RUN_OPTIONS
)


Starting Evaluation: colab_hetero18_1767404215
Results saved to: /content/slm_evaluation/results/colab_hetero18_1767404215/outputs.jsonl
Text log saved to: /content/slm_evaluation/results/colab_hetero18_1767404215/run_log.txt
Models: 18 | concurrency=6 | samples=819
--------------------------------------------------------------------------------


Processing Samples:   0%|          | 0/819 [00:00<?, ?it/s]


SAMPLE 1/819 | ID: samsum_test_0000
MESSAGE (preview): Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda:  ...[truncated]
------------------------------------------------------------------------------------------
MODEL 01: qwen2.5:7b-instruct
  status: OK | latency: 9.62s | attempts: 1
  summary: Hannah reluctantly agrees to ask Larry for Betty's number after Amanda suggests it.

MODEL 02: llama3.1:8b
  status: OK | latency: 25.96s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

MODEL 03: mistral:7b-instruct
  status: OK | latency: 6.69s | attempts: 1
  summary: Hannah asks Am

CancelledError: 

In [65]:
!pkill -f "ollama serve" || true
!nohup env \
  OLLAMA_MODELS="/content/" \
  OLLAMA_KEEP_ALIVE="24h" \
  OLLAMA_MAX_LOADED_MODELS="18" \
  OLLAMA_NUM_PARALLEL="16" \
  OLLAMA_MAX_QUEUE="2048" \
  ollama serve > /tmp/ollama.log 2>&1 &


^C


In [68]:
!pkill -f "ollama serve" || true
!nohup env \
  OLLAMA_MODELS="/content/" \
  OLLAMA_KEEP_ALIVE="24h" \
  OLLAMA_MAX_LOADED_MODELS="18" \
  OLLAMA_NUM_PARALLEL="16" \
  OLLAMA_MAX_QUEUE="2048" \
  ollama serve > /tmp/ollama.log 2>&1 &



import os, time, json, asyncio
from pathlib import Path

# Assumes these already exist in your notebook:
# - DATASET_PATH
# - RESULTS_DIR
# - RUN_OPTIONS
# - call_ollama(model, prompt, client, options) -> (txt, latency)
# - ollama_is_up() -> bool
# - get_consensus(list_of_strings) -> (best_idx, meta)
# - check_factuality(message, summary) -> any
# Also requires: httpx, tqdm

def _truncate(s: str, n: int = 400):
    if s is None:
        return ""
    s = str(s)
    return s if len(s) <= n else s[:n] + " ...[truncated]"

def _format_candidate_block(c, idx=None):
    header = f"AGENT {idx+1:02d}: {c['model']}" if idx is not None else f"MODEL: {c['model']}"
    lat = f"{c.get('latency', 0.0):.2f}s"
    att = c.get("attempts", 1)
    status = "OK" if not str(c.get("summary","")).startswith("ERROR") else "ERROR"
    return (
        f"{header}\n"
        f"  status: {status} | latency: {lat} | attempts: {att}\n"
        f"  summary: {c.get('summary','')}\n"
    )

async def call_with_retry(model, prompt, client, options, retries=3):
    last_txt, last_lat = None, None
    for k in range(1, retries + 1):
        txt, lat = await call_ollama(model, prompt, client, options)
        if isinstance(txt, str) and not txt.startswith("ERROR"):
            return txt, lat, k
        last_txt, last_lat = txt, lat
        await asyncio.sleep(0.5 * k)
    return last_txt, last_lat, retries

async def run_eval_homo6(
    model="llama3.1:8b",
    max_samples=None,
    agents=6,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    write_text_log=True,
    options=RUN_OPTIONS,
):
    import httpx
    from tqdm import tqdm

    if not os.path.exists(DATASET_PATH):
        print(f"ERROR: Dataset not found at {DATASET_PATH}. Upload samsum.jsonl to /content/")
        return

    if not await ollama_is_up():
        print("ERROR: Ollama not reachable at http://127.0.0.1:11434")
        print("Try: !tail -n 120 /tmp/ollama.log")
        return

    dataset = []
    with open(DATASET_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(json.loads(line))

    if max_samples is not None:
        dataset = dataset[:max_samples]

    run_id = f"colab_homo6_{model.replace(':','_').replace('/','_')}_{int(time.time())}"
    run_dir = Path(RESULTS_DIR) / run_id
    run_dir.mkdir(parents=True, exist_ok=True)
    out_file = run_dir / "outputs.jsonl"
    log_file = run_dir / "run_log.txt"

    print(f"Starting Homogeneous Evaluation: {run_id}")
    print(f"Results saved to: {out_file}")
    if write_text_log:
        print(f"Text log saved to: {log_file}")
    print(f"Model: {model} | agents={agents} | concurrency={concurrency} | samples={len(dataset)}")
    print("-" * 80)

    sem = asyncio.Semaphore(concurrency)

    def _log_line(s: str):
        if write_text_log:
            with open(log_file, "a", encoding="utf-8") as lf:
                lf.write(s + "\n")

    async with httpx.AsyncClient(timeout=300.0) as client:
        for idx_item, item in enumerate(tqdm(dataset, desc="Processing Samples")):
            prompt = f"Summarize this message in one short sentence:\n\n{item['message']}\n\nSummary:"

            async def _one(agent_idx: int):
                async with sem:
                    txt, lat, attempts = await call_with_retry(model, prompt, client, options, retries=3)
                    return agent_idx, txt, lat, attempts

            results = await asyncio.gather(*[_one(i) for i in range(agents)])

            candidates = []
            for (agent_idx, txt, lat, attempts) in results:
                candidates.append({
                    "model": model,
                    "agent_idx": int(agent_idx),
                    "summary": txt,
                    "latency": float(lat),
                    "attempts": int(attempts),
                })

            valid_summaries = [
                c["summary"] for c in candidates
                if isinstance(c["summary"], str) and not c["summary"].startswith("ERROR")
            ]

            if valid_summaries:
                best_idx_valid, meta = get_consensus(valid_summaries)
                swarm_summary = valid_summaries[best_idx_valid]
            else:
                meta = {}
                swarm_summary = "ERROR: All agents failed"

            do_print = print_each and ((idx_item % print_every) == 0)
            if do_print:
                sep = "=" * 90
                header = f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}"
                msg_preview = _truncate(item["message"].replace("\n", " "), 400)

                print("\n" + sep)
                print(header)
                print("MESSAGE (preview):", msg_preview)
                print("-" * 90)

                _log_line("\n" + sep)
                _log_line(header)
                _log_line("MESSAGE:")
                _log_line(item["message"])
                _log_line("-" * 90)

                for j, c in enumerate(candidates):
                    c_out = dict(c)
                    if truncate_console and isinstance(c_out["summary"], str):
                        c_out["summary"] = _truncate(c_out["summary"], truncate_console)
                    block = _format_candidate_block(c_out, idx=j)
                    print(block)
                    _log_line(_format_candidate_block(c, idx=j))

                print("-" * 90)
                print("CONSENSUS avg_similarity:", meta.get("avg_similarity"))
                if meta.get("consensus_scores") is not None:
                    print("CONSENSUS scores:", meta.get("consensus_scores"))
                print("SWARM SUMMARY:", swarm_summary)
                print(sep + "\n")

                _log_line("-" * 90)
                _log_line(f"CONSENSUS avg_similarity: {meta.get('avg_similarity')}")
                _log_line(f"CONSENSUS scores: {meta.get('consensus_scores')}")
                _log_line(f"SWARM SUMMARY: {swarm_summary}")
                _log_line(sep)

            result = {
                "id": item.get("id"),
                "model": model,
                "agents": int(agents),
                "swarm_summary": swarm_summary,
                "swarm_candidates": candidates,
                "consensus_metadata": meta,
                "factuality": check_factuality(item["message"], swarm_summary),
            }

            with open(out_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(result) + "\n")

    print(f"\n✓ Done. outputs.jsonl: {out_file}")
    if write_text_log:
        print(f"✓ Done. run_log.txt:  {log_file}")

# Run (default: llama3.1:8b, 6 agents)
await run_eval_homo6(
    model="llama3.1:8b",
    max_samples=None,
    agents=6,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    options=RUN_OPTIONS
)


^C
Starting Homogeneous Evaluation: colab_homo6_llama3.1_8b_1767405547
Results saved to: /content/slm_evaluation/results/colab_homo6_llama3.1_8b_1767405547/outputs.jsonl
Text log saved to: /content/slm_evaluation/results/colab_homo6_llama3.1_8b_1767405547/run_log.txt
Model: llama3.1:8b | agents=6 | concurrency=6 | samples=819
--------------------------------------------------------------------------------


Processing Samples:   0%|          | 1/819 [00:03<52:25,  3.85s/it]


SAMPLE 1/819 | ID: samsum_test_0000
MESSAGE (preview): Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda:  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 3.75s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

AGENT 02: llama3.1:8b
  status: OK | latency: 3.69s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

AGENT 03: llama3.1:8b
  status: OK | latency: 3.74s | attempts: 1
  summary: Hannah asked Amanda for Be

Processing Samples:   1%|▏         | 11/819 [00:17<17:33,  1.30s/it]


SAMPLE 11/819 | ID: samsum_test_0010
MESSAGE (preview): Wanda: Let's make a party! Gina: Why? Wanda: beacuse. I want some fun! Gina: ok, what do u need? Wanda: 1st I need too make a list Gina: noted and then? Wanda: well, could u take yours father car and go do groceries with me? Gina: don't know if he'll agree Wanda: I know, but u can ask :) Gina: I'll try but theres no promisess Wanda: I know, u r the best! Gina: When u wanna go Wanda: Friday? Gina:  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.13s | attempts: 1
  summary: Wanda wants to have a party and asks Gina for help with errands so they can plan it together.

AGENT 02: llama3.1:8b
  status: OK | latency: 1.04s | attempts: 1
  summary: Wanda wants to have a party and needs Gina's help with errands before it can happen.

AGENT 03: llama3.1:8b
  status: OK | latency: 1.10s | attempts: 1
  summary: Wanda wants to have a pa

...


SAMPLE 801/819 | ID: samsum_test_0800
MESSAGE (preview): Linda: Hi Dad, I want to buy flowers for mum! But I don't remember which one she likes :( Michael: Well, she likes all the flowers I believe Linda: That doesn't help! I'm on a flower market right now! Michael: Send me some pics then Linda: <file_photo>  Michael: Tulips are nice, roses too Linda:  What about carnations? Michael: No, carnations are boring :D Linda: Thanks Dad, srsly… Michael:  What  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.26s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mom and he guides her through the selection process via photos.

AGENT 02: llama3.1:8b
  status: OK | latency: 1.26s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mother and he guides her through the selection process via photos.

AGENT 03: llama3.1:8b
  

Processing Samples:  99%|█████████▉| 811/819 [17:10<00:09,  1.13s/it]


SAMPLE 811/819 | ID: samsum_test_0810
MESSAGE (preview): Luke: Ryanair to cancel flights affecting 400,000 more passengers. So unlucky Erin! Erin: Yeeeep...it touched me as well finally 😩😩😩😩 Now they screw all my plans to go to Egypt... Luke: You’re not the only one Erin: Good to know that my feelings are shared with other 399,999 people 😩 Nathan: Relevant news story for everyone to consider: http://waterfordwhispersnews.com/2017/09/18/ryanair-cancel-ef ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.14s | attempts: 1
  summary: Ryanair has announced it will cancel more flights, affecting over 400,000 passengers.

AGENT 02: llama3.1:8b
  status: OK | latency: 0.99s | attempts: 1
  summary: Ryanair has cancelled flights affecting over 400,000 passengers.

AGENT 03: llama3.1:8b
  status: OK | latency: 1.14s | attempts: 1
  summary: Ryanair has announced it will cancel more flights, 

Processing Samples: 100%|██████████| 819/819 [17:21<00:00,  1.27s/it]


✓ Done. outputs.jsonl: /content/slm_evaluation/results/colab_homo6_llama3.1_8b_1767405547/outputs.jsonl
✓ Done. run_log.txt:  /content/slm_evaluation/results/colab_homo6_llama3.1_8b_1767405547/run_log.txt


In [69]:
!pkill -f "ollama serve" || true
!nohup env \
  OLLAMA_MODELS="/content/" \
  OLLAMA_KEEP_ALIVE="24h" \
  OLLAMA_MAX_LOADED_MODELS="18" \
  OLLAMA_NUM_PARALLEL="16" \
  OLLAMA_MAX_QUEUE="2048" \
  ollama serve > /tmp/ollama.log 2>&1 &

import os, time, json, asyncio
from pathlib import Path

HETRO6_MODELS = [
    "llama3.1:8b",
    "qwen2.5:7b-instruct",
    "mistral:7b-instruct",
    "gemma:7b-instruct",
    "openchat:7b",
    "neural-chat:7b",
]

def _truncate(s: str, n: int = 400):
    if s is None:
        return ""
    s = str(s)
    return s if len(s) <= n else s[:n] + " ...[truncated]"

def _format_candidate_block(c, idx=None):
    header = f"MODEL {idx+1:02d}: {c['model']}" if idx is not None else f"MODEL: {c['model']}"
    lat = f"{c.get('latency', 0.0):.2f}s"
    att = c.get("attempts", 1)
    status = "OK" if not str(c.get("summary","")).startswith("ERROR") else "ERROR"
    return (
        f"{header}\n"
        f"  status: {status} | latency: {lat} | attempts: {att}\n"
        f"  summary: {c.get('summary','')}\n"
    )

async def call_with_retry(model, prompt, client, options, retries=3):
    last_txt, last_lat = None, None
    for k in range(1, retries + 1):
        txt, lat = await call_ollama(model, prompt, client, options)
        if isinstance(txt, str) and not txt.startswith("ERROR"):
            return txt, lat, k
        last_txt, last_lat = txt, lat
        await asyncio.sleep(0.5 * k)
    return last_txt, last_lat, retries

async def run_eval_hetero6(
    max_samples=None,
    models=HETRO6_MODELS,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    write_text_log=True,
    options=RUN_OPTIONS,
):
    import httpx
    from tqdm import tqdm

    if not os.path.exists(DATASET_PATH):
        print(f"ERROR: Dataset not found at {DATASET_PATH}. Upload samsum.jsonl to /content/")
        return

    if not await ollama_is_up():
        print("ERROR: Ollama not reachable at http://127.0.0.1:11434")
        print("Try: !tail -n 120 /tmp/ollama.log")
        return

    dataset = []
    with open(DATASET_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(json.loads(line))

    if max_samples is not None:
        dataset = dataset[:max_samples]

    run_id = f"colab_hetero6_highparams_{int(time.time())}"
    run_dir = Path(RESULTS_DIR) / run_id
    run_dir.mkdir(parents=True, exist_ok=True)
    out_file = run_dir / "outputs.jsonl"
    log_file = run_dir / "run_log.txt"

    print(f"Starting Heterogeneous-6 Evaluation: {run_id}")
    print(f"Results saved to: {out_file}")
    if write_text_log:
        print(f"Text log saved to: {log_file}")
    print(f"Models: {len(models)} | concurrency={concurrency} | samples={len(dataset)}")
    print("Models:", models)
    print("-" * 80)

    sem = asyncio.Semaphore(concurrency)

    def _log_line(s: str):
        if write_text_log:
            with open(log_file, "a", encoding="utf-8") as lf:
                lf.write(s + "\n")

    async with httpx.AsyncClient(timeout=300.0) as client:
        for idx_item, item in enumerate(tqdm(dataset, desc="Processing Samples")):
            prompt = f"Summarize this message in one short sentence:\n\n{item['message']}\n\nSummary:"

            async def _one(m):
                async with sem:
                    txt, lat, attempts = await call_with_retry(m, prompt, client, options, retries=3)
                    return m, txt, lat, attempts

            results = await asyncio.gather(*[_one(m) for m in models])

            candidates = []
            for (m, txt, lat, attempts) in results:
                candidates.append({
                    "model": m,
                    "summary": txt,
                    "latency": float(lat),
                    "attempts": int(attempts),
                })

            valid_summaries = [
                c["summary"] for c in candidates
                if isinstance(c["summary"], str) and not c["summary"].startswith("ERROR")
            ]

            if valid_summaries:
                best_idx_valid, meta = get_consensus(valid_summaries)
                swarm_summary = valid_summaries[best_idx_valid]
            else:
                meta = {}
                swarm_summary = "ERROR: All models failed"

            do_print = print_each and ((idx_item % print_every) == 0)
            if do_print:
                sep = "=" * 90
                header = f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}"
                msg_preview = _truncate(item["message"].replace("\n", " "), 400)

                print("\n" + sep)
                print(header)
                print("MESSAGE (preview):", msg_preview)
                print("-" * 90)

                _log_line("\n" + sep)
                _log_line(header)
                _log_line("MESSAGE:")
                _log_line(item["message"])
                _log_line("-" * 90)

                for j, c in enumerate(candidates):
                    c_out = dict(c)
                    if truncate_console and isinstance(c_out["summary"], str):
                        c_out["summary"] = _truncate(c_out["summary"], truncate_console)
                    block = _format_candidate_block(c_out, idx=j)
                    print(block)
                    _log_line(_format_candidate_block(c, idx=j))

                print("-" * 90)
                print("CONSENSUS avg_similarity:", meta.get("avg_similarity"))
                if meta.get("consensus_scores") is not None:
                    print("CONSENSUS scores:", meta.get("consensus_scores"))
                print("SWARM SUMMARY:", swarm_summary)
                print(sep + "\n")

                _log_line("-" * 90)
                _log_line(f"CONSENSUS avg_similarity: {meta.get('avg_similarity')}")
                _log_line(f"CONSENSUS scores: {meta.get('consensus_scores')}")
                _log_line(f"SWARM SUMMARY: {swarm_summary}")
                _log_line(sep)

            result = {
                "id": item.get("id"),
                "swarm_summary": swarm_summary,
                "swarm_candidates": candidates,
                "consensus_metadata": meta,
                "factuality": check_factuality(item["message"], swarm_summary),
                "models": models,
            }

            with open(out_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(result) + "\n")

    print(f"\n✓ Done. outputs.jsonl: {out_file}")
    if write_text_log:
        print(f"✓ Done. run_log.txt:  {log_file}")

# Run
await run_eval_hetero6(
    max_samples=None,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    options=RUN_OPTIONS
)


^C
Starting Heterogeneous-6 Evaluation: colab_hetero6_highparams_1767406589
Results saved to: /content/slm_evaluation/results/colab_hetero6_highparams_1767406589/outputs.jsonl
Text log saved to: /content/slm_evaluation/results/colab_hetero6_highparams_1767406589/run_log.txt
Models: 6 | concurrency=6 | samples=819
Models: ['llama3.1:8b', 'qwen2.5:7b-instruct', 'mistral:7b-instruct', 'gemma:7b-instruct', 'openchat:7b', 'neural-chat:7b']
--------------------------------------------------------------------------------


Processing Samples:   0%|          | 1/819 [00:20<4:41:47, 20.67s/it]


SAMPLE 1/819 | ID: samsum_test_0000
MESSAGE (preview): Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda:  ...[truncated]
------------------------------------------------------------------------------------------
MODEL 01: llama3.1:8b
  status: OK | latency: 20.63s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

MODEL 02: qwen2.5:7b-instruct
  status: OK | latency: 8.35s | attempts: 1
  summary: Hannah reluctantly agrees to ask Larry for Betty's number after Amanda suggests it.

MODEL 03: mistral:7b-instruct
  status: OK | latency: 2.68s | attempts: 1
  summary: Hannah asks Am

Processing Samples:   1%|▏         | 11/819 [02:02<2:17:33, 10.22s/it]


SAMPLE 11/819 | ID: samsum_test_0010
MESSAGE (preview): Wanda: Let's make a party! Gina: Why? Wanda: beacuse. I want some fun! Gina: ok, what do u need? Wanda: 1st I need too make a list Gina: noted and then? Wanda: well, could u take yours father car and go do groceries with me? Gina: don't know if he'll agree Wanda: I know, but u can ask :) Gina: I'll try but theres no promisess Wanda: I know, u r the best! Gina: When u wanna go Wanda: Friday? Gina:  ...[truncated]
------------------------------------------------------------------------------------------
MODEL 01: llama3.1:8b
  status: OK | latency: 0.93s | attempts: 1
  summary: Wanda wants to have a party and needs Gina's help with errands before it can happen.

MODEL 02: qwen2.5:7b-instruct
  status: OK | latency: 0.91s | attempts: 1
  summary: Wanda is planning a party and asks Gina to help by going grocery shopping and inviting their father's car.

MODEL 03: mistral:7b-instruct
  status: OK | latency: 0.80s | attempts: 1
  summ

...


SAMPLE 801/819 | ID: samsum_test_0800
MESSAGE (preview): Linda: Hi Dad, I want to buy flowers for mum! But I don't remember which one she likes :( Michael: Well, she likes all the flowers I believe Linda: That doesn't help! I'm on a flower market right now! Michael: Send me some pics then Linda: <file_photo>  Michael: Tulips are nice, roses too Linda:  What about carnations? Michael: No, carnations are boring :D Linda: Thanks Dad, srsly… Michael:  What  ...[truncated]
------------------------------------------------------------------------------------------
MODEL 01: llama3.1:8b
  status: OK | latency: 0.80s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mother and he guides her through the options.

MODEL 02: qwen2.5:7b-instruct
  status: OK | latency: 8.16s | attempts: 1
  summary: Linda is getting flowers for her mom and gets help from her dad to choose the right type.

MODEL 03: mistral:7b-instruct
  status: OK | latency: 0.86s | attempts: 1

Processing Samples:  99%|█████████▉| 811/819 [2:20:55<01:18,  9.78s/it]


SAMPLE 811/819 | ID: samsum_test_0810
MESSAGE (preview): Luke: Ryanair to cancel flights affecting 400,000 more passengers. So unlucky Erin! Erin: Yeeeep...it touched me as well finally 😩😩😩😩 Now they screw all my plans to go to Egypt... Luke: You’re not the only one Erin: Good to know that my feelings are shared with other 399,999 people 😩 Nathan: Relevant news story for everyone to consider: http://waterfordwhispersnews.com/2017/09/18/ryanair-cancel-ef ...[truncated]
------------------------------------------------------------------------------------------
MODEL 01: llama3.1:8b
  status: OK | latency: 0.67s | attempts: 1
  summary: Ryanair has cancelled flights affecting over 400,000 passengers.

MODEL 02: qwen2.5:7b-instruct
  status: OK | latency: 8.38s | attempts: 1
  summary: Many passengers, including Erin and Ava, are facing flight cancellations by Ryanair, leading to disrupted travel plans and potential compensation hopes.

MODEL 03: mistral:7b-instruct
  status: OK | latency:

Processing Samples: 100%|██████████| 819/819 [2:22:19<00:00, 10.43s/it]


✓ Done. outputs.jsonl: /content/slm_evaluation/results/colab_hetero6_highparams_1767406589/outputs.jsonl
✓ Done. run_log.txt:  /content/slm_evaluation/results/colab_hetero6_highparams_1767406589/run_log.txt


Below for 12 homo

In [71]:
!pkill -f "ollama serve" || true
!nohup env \
  OLLAMA_MODELS="/content/" \
  OLLAMA_KEEP_ALIVE="24h" \
  OLLAMA_MAX_LOADED_MODELS="18" \
  OLLAMA_NUM_PARALLEL="16" \
  OLLAMA_MAX_QUEUE="2048" \
  ollama serve > /tmp/ollama.log 2>&1 &

import os, time, json, asyncio
from pathlib import Path

# Assumes these already exist in your notebook:
# - DATASET_PATH
# - RESULTS_DIR
# - RUN_OPTIONS
# - call_ollama(model, prompt, client, options) -> (txt, latency)
# - ollama_is_up() -> bool
# - get_consensus(list_of_strings) -> (best_idx, meta)
# - check_factuality(message, summary) -> any
# Also requires: httpx, tqdm

def _truncate(s: str, n: int = 400):
    if s is None:
        return ""
    s = str(s)
    return s if len(s) <= n else s[:n] + " ...[truncated]"

def _format_candidate_block(c, idx=None):
    header = f"AGENT {idx+1:02d}: {c['model']}" if idx is not None else f"MODEL: {c['model']}"
    lat = f"{c.get('latency', 0.0):.2f}s"
    att = c.get("attempts", 1)
    status = "OK" if not str(c.get("summary","")).startswith("ERROR") else "ERROR"
    return (
        f"{header}\n"
        f"  status: {status} | latency: {lat} | attempts: {att}\n"
        f"  summary: {c.get('summary','')}\n"
    )

async def call_with_retry(model, prompt, client, options, retries=3):
    last_txt, last_lat = None, None
    for k in range(1, retries + 1):
        txt, lat = await call_ollama(model, prompt, client, options)
        if isinstance(txt, str) and not txt.startswith("ERROR"):
            return txt, lat, k
        last_txt, last_lat = txt, lat
        await asyncio.sleep(0.5 * k)
    return last_txt, last_lat, retries

async def run_eval_homo12(
    model="llama3.1:8b",
    max_samples=None,
    agents=12,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    write_text_log=True,
    options=RUN_OPTIONS,
):
    import httpx
    from tqdm import tqdm

    if not os.path.exists(DATASET_PATH):
        print(f"ERROR: Dataset not found at {DATASET_PATH}. Upload samsum.jsonl to /content/")
        return

    if not await ollama_is_up():
        print("ERROR: Ollama not reachable at http://127.0.0.1:11434")
        print("Try: !tail -n 120 /tmp/ollama.log")
        return

    dataset = []
    with open(DATASET_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(json.loads(line))

    if max_samples is not None:
        dataset = dataset[:max_samples]

    run_id = f"colab_homo12_{model.replace(':','_').replace('/','_')}_{int(time.time())}"
    run_dir = Path(RESULTS_DIR) / run_id
    run_dir.mkdir(parents=True, exist_ok=True)
    out_file = run_dir / "outputs.jsonl"
    log_file = run_dir / "run_log.txt"

    print(f"Starting Homogeneous-12 Evaluation: {run_id}")
    print(f"Results saved to: {out_file}")
    if write_text_log:
        print(f"Text log saved to: {log_file}")
    print(f"Model: {model} | agents={agents} | concurrency={concurrency} | samples={len(dataset)}")
    print("-" * 80)

    sem = asyncio.Semaphore(concurrency)

    def _log_line(s: str):
        if write_text_log:
            with open(log_file, "a", encoding="utf-8") as lf:
                lf.write(s + "\n")

    async with httpx.AsyncClient(timeout=300.0) as client:
        for idx_item, item in enumerate(tqdm(dataset, desc="Processing Samples")):
            prompt = f"Summarize this message in one short sentence:\n\n{item['message']}\n\nSummary:"

            async def _one(agent_idx: int):
                async with sem:
                    txt, lat, attempts = await call_with_retry(model, prompt, client, options, retries=3)
                    return agent_idx, txt, lat, attempts

            results = await asyncio.gather(*[_one(i) for i in range(agents)])

            candidates = []
            for (agent_idx, txt, lat, attempts) in results:
                candidates.append({
                    "model": model,
                    "agent_idx": int(agent_idx),
                    "summary": txt,
                    "latency": float(lat),
                    "attempts": int(attempts),
                })

            valid_summaries = [
                c["summary"] for c in candidates
                if isinstance(c["summary"], str) and not c["summary"].startswith("ERROR")
            ]

            if valid_summaries:
                best_idx_valid, meta = get_consensus(valid_summaries)
                swarm_summary = valid_summaries[best_idx_valid]
            else:
                meta = {}
                swarm_summary = "ERROR: All agents failed"

            do_print = print_each and ((idx_item % print_every) == 0)
            if do_print:
                sep = "=" * 90
                header = f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}"
                msg_preview = _truncate(item["message"].replace("\n", " "), 400)

                print("\n" + sep)
                print(header)
                print("MESSAGE (preview):", msg_preview)
                print("-" * 90)

                _log_line("\n" + sep)
                _log_line(header)
                _log_line("MESSAGE:")
                _log_line(item["message"])
                _log_line("-" * 90)

                for j, c in enumerate(candidates):
                    c_out = dict(c)
                    if truncate_console and isinstance(c_out["summary"], str):
                        c_out["summary"] = _truncate(c_out["summary"], truncate_console)
                    block = _format_candidate_block(c_out, idx=j)
                    print(block)
                    _log_line(_format_candidate_block(c, idx=j))

                print("-" * 90)
                print("CONSENSUS avg_similarity:", meta.get("avg_similarity"))
                if meta.get("consensus_scores") is not None:
                    print("CONSENSUS scores:", meta.get("consensus_scores"))
                print("SWARM SUMMARY:", swarm_summary)
                print(sep + "\n")

                _log_line("-" * 90)
                _log_line(f"CONSENSUS avg_similarity: {meta.get('avg_similarity')}")
                _log_line(f"CONSENSUS scores: {meta.get('consensus_scores')}")
                _log_line(f"SWARM SUMMARY: {swarm_summary}")
                _log_line(sep)

            result = {
                "id": item.get("id"),
                "model": model,
                "agents": int(agents),
                "swarm_summary": swarm_summary,
                "swarm_candidates": candidates,
                "consensus_metadata": meta,
                "factuality": check_factuality(item["message"], swarm_summary),
            }

            with open(out_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(result) + "\n")

    print(f"\n✓ Done. outputs.jsonl: {out_file}")
    if write_text_log:
        print(f"✓ Done. run_log.txt:  {log_file}")

# Run (default: llama3.1:8b, 12 agents)
await run_eval_homo12(
    model="llama3.1:8b",
    max_samples=None,
    agents=12,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    options=RUN_OPTIONS
)


^C
Starting Homogeneous-12 Evaluation: colab_homo12_llama3.1_8b_1767415147
Results saved to: /content/slm_evaluation/results/colab_homo12_llama3.1_8b_1767415147/outputs.jsonl
Text log saved to: /content/slm_evaluation/results/colab_homo12_llama3.1_8b_1767415147/run_log.txt
Model: llama3.1:8b | agents=12 | concurrency=6 | samples=819
--------------------------------------------------------------------------------


Processing Samples:   0%|          | 1/819 [00:04<1:02:52,  4.61s/it]


SAMPLE 1/819 | ID: samsum_test_0000
MESSAGE (preview): Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda:  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 3.78s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

AGENT 02: llama3.1:8b
  status: OK | latency: 3.72s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

AGENT 03: llama3.1:8b
  status: OK | latency: 3.78s | attempts: 1
  summary: Hannah asked Amanda for Be

Processing Samples:   1%|▏         | 11/819 [00:27<29:01,  2.16s/it]


SAMPLE 11/819 | ID: samsum_test_0010
MESSAGE (preview): Wanda: Let's make a party! Gina: Why? Wanda: beacuse. I want some fun! Gina: ok, what do u need? Wanda: 1st I need too make a list Gina: noted and then? Wanda: well, could u take yours father car and go do groceries with me? Gina: don't know if he'll agree Wanda: I know, but u can ask :) Gina: I'll try but theres no promisess Wanda: I know, u r the best! Gina: When u wanna go Wanda: Friday? Gina:  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.09s | attempts: 1
  summary: Wanda wants to have a party and needs Gina's help with errands before it can happen.

AGENT 02: llama3.1:8b
  status: OK | latency: 1.09s | attempts: 1
  summary: Wanda wants to have a party and needs Gina's help with errands before it can happen.

AGENT 03: llama3.1:8b
  status: OK | latency: 1.09s | attempts: 1
  summary: Wanda wants to have a party and n

...


SAMPLE 801/819 | ID: samsum_test_0800
MESSAGE (preview): Linda: Hi Dad, I want to buy flowers for mum! But I don't remember which one she likes :( Michael: Well, she likes all the flowers I believe Linda: That doesn't help! I'm on a flower market right now! Michael: Send me some pics then Linda: <file_photo>  Michael: Tulips are nice, roses too Linda:  What about carnations? Michael: No, carnations are boring :D Linda: Thanks Dad, srsly… Michael:  What  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.15s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mother and he guides her through the options.

AGENT 02: llama3.1:8b
  status: OK | latency: 1.35s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mother and he guides her through the process by suggesting specific types of flowers.

AGENT 03: llama3.1:8b
 

Processing Samples:  99%|█████████▉| 811/819 [29:36<00:15,  1.97s/it]


SAMPLE 811/819 | ID: samsum_test_0810
MESSAGE (preview): Luke: Ryanair to cancel flights affecting 400,000 more passengers. So unlucky Erin! Erin: Yeeeep...it touched me as well finally 😩😩😩😩 Now they screw all my plans to go to Egypt... Luke: You’re not the only one Erin: Good to know that my feelings are shared with other 399,999 people 😩 Nathan: Relevant news story for everyone to consider: http://waterfordwhispersnews.com/2017/09/18/ryanair-cancel-ef ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.00s | attempts: 1
  summary: Ryanair has cancelled flights affecting over 400,000 passengers.

AGENT 02: llama3.1:8b
  status: OK | latency: 1.01s | attempts: 1
  summary: Ryanair has cancelled flights affecting over 400,000 passengers.

AGENT 03: llama3.1:8b
  status: OK | latency: 1.08s | attempts: 1
  summary: Ryanair has announced it will cancel more flights, affecting around 400,

Processing Samples: 100%|██████████| 819/819 [29:55<00:00,  2.19s/it]


✓ Done. outputs.jsonl: /content/slm_evaluation/results/colab_homo12_llama3.1_8b_1767415147/outputs.jsonl
✓ Done. run_log.txt:  /content/slm_evaluation/results/colab_homo12_llama3.1_8b_1767415147/run_log.txt


12 hetro

In [70]:
!pkill -f "ollama serve" || true
!nohup env \
  OLLAMA_MODELS="/content/" \
  OLLAMA_KEEP_ALIVE="24h" \
  OLLAMA_MAX_LOADED_MODELS="18" \
  OLLAMA_NUM_PARALLEL="16" \
  OLLAMA_MAX_QUEUE="2048" \
  ollama serve > /tmp/ollama.log 2>&1 &

import os, time, json, asyncio
from pathlib import Path

# Picks the 12 highest-parameter models from YOUR MODELS list (7B/8B only)
HETRO12_MODELS = [
    "llama3.1:8b",
    "qwen2.5:7b-instruct",
    "mistral:7b-instruct",
    "gemma:7b-instruct",
    "openchat:7b",
    "neural-chat:7b",
    "dolphin-mistral:7b",
    "zephyr:7b",
    "orca-mini:7b",
    "phi3:medium",
    "gemma:7b-instruct",   # duplicate protection note below
    "qwen2.5:7b-instruct", # duplicate protection note below
]

# Remove duplicates while preserving order, then enforce exactly 12 if possible
_seen = set()
HETRO12_MODELS = [m for m in HETRO12_MODELS if not (m in _seen or _seen.add(m))]

# If your list had fewer than 12 unique "high-param" models, backfill from your MODELS (largest-first)
if len(HETRO12_MODELS) < 12:
    preferred = [
        "stablelm2:1.6b",
        "tinyllama:1.1b",
        "llama3.2:3b",
        "qwen2.5:3b-instruct",
        "gemma:2b",
        "qwen2.5:1.5b-instruct",
        "llama3.2:1b",
        "phi3:mini",
    ]
    for m in preferred:
        if m not in HETRO12_MODELS:
            HETRO12_MODELS.append(m)
        if len(HETRO12_MODELS) == 12:
            break

assert len(HETRO12_MODELS) == 12, f"Need 12 models, got {len(HETRO12_MODELS)}: {HETRO12_MODELS}"

def _truncate(s: str, n: int = 400):
    if s is None:
        return ""
    s = str(s)
    return s if len(s) <= n else s[:n] + " ...[truncated]"

def _format_candidate_block(c, idx=None):
    header = f"MODEL {idx+1:02d}: {c['model']}" if idx is not None else f"MODEL: {c['model']}"
    lat = f"{c.get('latency', 0.0):.2f}s"
    att = c.get("attempts", 1)
    status = "OK" if not str(c.get("summary","")).startswith("ERROR") else "ERROR"
    return (
        f"{header}\n"
        f"  status: {status} | latency: {lat} | attempts: {att}\n"
        f"  summary: {c.get('summary','')}\n"
    )

async def call_with_retry(model, prompt, client, options, retries=3):
    last_txt, last_lat = None, None
    for k in range(1, retries + 1):
        txt, lat = await call_ollama(model, prompt, client, options)
        if isinstance(txt, str) and not txt.startswith("ERROR"):
            return txt, lat, k
        last_txt, last_lat = txt, lat
        await asyncio.sleep(0.5 * k)
    return last_txt, last_lat, retries

async def run_eval_hetero12(
    max_samples=None,
    models=HETRO12_MODELS,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    write_text_log=True,
    options=RUN_OPTIONS,
):
    import httpx
    from tqdm import tqdm

    if not os.path.exists(DATASET_PATH):
        print(f"ERROR: Dataset not found at {DATASET_PATH}. Upload samsum.jsonl to /content/")
        return

    if not await ollama_is_up():
        print("ERROR: Ollama not reachable at http://127.0.0.1:11434")
        print("Try: !tail -n 120 /tmp/ollama.log")
        return

    dataset = []
    with open(DATASET_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(json.loads(line))

    if max_samples is not None:
        dataset = dataset[:max_samples]

    run_id = f"colab_hetero12_{int(time.time())}"
    run_dir = Path(RESULTS_DIR) / run_id
    run_dir.mkdir(parents=True, exist_ok=True)
    out_file = run_dir / "outputs.jsonl"
    log_file = run_dir / "run_log.txt"

    print(f"Starting Heterogeneous-12 Evaluation: {run_id}")
    print(f"Results saved to: {out_file}")
    if write_text_log:
        print(f"Text log saved to: {log_file}")
    print(f"Models: {len(models)} | concurrency={concurrency} | samples={len(dataset)}")
    print("Models:", models)
    print("-" * 80)

    sem = asyncio.Semaphore(concurrency)

    def _log_line(s: str):
        if write_text_log:
            with open(log_file, "a", encoding="utf-8") as lf:
                lf.write(s + "\n")

    async with httpx.AsyncClient(timeout=300.0) as client:
        for idx_item, item in enumerate(tqdm(dataset, desc="Processing Samples")):
            prompt = f"Summarize this message in one short sentence:\n\n{item['message']}\n\nSummary:"

            async def _one(m):
                async with sem:
                    txt, lat, attempts = await call_with_retry(m, prompt, client, options, retries=3)
                    return m, txt, lat, attempts

            results = await asyncio.gather(*[_one(m) for m in models])

            candidates = []
            for (m, txt, lat, attempts) in results:
                candidates.append({
                    "model": m,
                    "summary": txt,
                    "latency": float(lat),
                    "attempts": int(attempts),
                })

            valid_summaries = [
                c["summary"] for c in candidates
                if isinstance(c["summary"], str) and not c["summary"].startswith("ERROR")
            ]

            if valid_summaries:
                best_idx_valid, meta = get_consensus(valid_summaries)
                swarm_summary = valid_summaries[best_idx_valid]
            else:
                meta = {}
                swarm_summary = "ERROR: All models failed"

            do_print = print_each and ((idx_item % print_every) == 0)
            if do_print:
                sep = "=" * 90
                header = f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}"
                msg_preview = _truncate(item["message"].replace("\n", " "), 400)

                print("\n" + sep)
                print(header)
                print("MESSAGE (preview):", msg_preview)
                print("-" * 90)

                _log_line("\n" + sep)
                _log_line(header)
                _log_line("MESSAGE:")
                _log_line(item["message"])
                _log_line("-" * 90)

                for j, c in enumerate(candidates):
                    c_out = dict(c)
                    if truncate_console and isinstance(c_out["summary"], str):
                        c_out["summary"] = _truncate(c_out["summary"], truncate_console)
                    block = _format_candidate_block(c_out, idx=j)
                    print(block)
                    _log_line(_format_candidate_block(c, idx=j))

                print("-" * 90)
                print("CONSENSUS avg_similarity:", meta.get("avg_similarity"))
                if meta.get("consensus_scores") is not None:
                    print("CONSENSUS scores:", meta.get("consensus_scores"))
                print("SWARM SUMMARY:", swarm_summary)
                print(sep + "\n")

                _log_line("-" * 90)
                _log_line(f"CONSENSUS avg_similarity: {meta.get('avg_similarity')}")
                _log_line(f"CONSENSUS scores: {meta.get('consensus_scores')}")
                _log_line(f"SWARM SUMMARY: {swarm_summary}")
                _log_line(sep)

            result = {
                "id": item.get("id"),
                "swarm_summary": swarm_summary,
                "swarm_candidates": candidates,
                "consensus_metadata": meta,
                "factuality": check_factuality(item["message"], swarm_summary),
                "models": models,
            }

            with open(out_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(result) + "\n")

    print(f"\n✓ Done. outputs.jsonl: {out_file}")
    if write_text_log:
        print(f"✓ Done. run_log.txt:  {log_file}")

# Run
await run_eval_hetero12(
    max_samples=None,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    options=RUN_OPTIONS
)


^C
Starting Heterogeneous-12 Evaluation: colab_hetero12_1767415130
Results saved to: /content/slm_evaluation/results/colab_hetero12_1767415130/outputs.jsonl
Text log saved to: /content/slm_evaluation/results/colab_hetero12_1767415130/run_log.txt
Models: 12 | concurrency=6 | samples=819
Models: ['llama3.1:8b', 'qwen2.5:7b-instruct', 'mistral:7b-instruct', 'gemma:7b-instruct', 'openchat:7b', 'neural-chat:7b', 'dolphin-mistral:7b', 'zephyr:7b', 'orca-mini:7b', 'phi3:medium', 'stablelm2:1.6b', 'tinyllama:1.1b']
--------------------------------------------------------------------------------


Processing Samples:   0%|          | 0/819 [00:13<?, ?it/s]


CancelledError: 

18 homo

In [72]:
!pkill -f "ollama serve" || true
!nohup env \
  OLLAMA_MODELS="/content/" \
  OLLAMA_KEEP_ALIVE="24h" \
  OLLAMA_MAX_LOADED_MODELS="18" \
  OLLAMA_NUM_PARALLEL="16" \
  OLLAMA_MAX_QUEUE="2048" \
  ollama serve > /tmp/ollama.log 2>&1 &

import os, time, json, asyncio
from pathlib import Path

# Assumes these already exist in your notebook:
# - DATASET_PATH
# - RESULTS_DIR
# - RUN_OPTIONS
# - call_ollama(model, prompt, client, options) -> (txt, latency)
# - ollama_is_up() -> bool
# - get_consensus(list_of_strings) -> (best_idx, meta)
# - check_factuality(message, summary) -> any
# Also requires: httpx, tqdm

def _truncate(s: str, n: int = 400):
    if s is None:
        return ""
    s = str(s)
    return s if len(s) <= n else s[:n] + " ...[truncated]"

def _format_candidate_block(c, idx=None):
    header = f"AGENT {idx+1:02d}: {c['model']}" if idx is not None else f"MODEL: {c['model']}"
    lat = f"{c.get('latency', 0.0):.2f}s"
    att = c.get("attempts", 1)
    status = "OK" if not str(c.get("summary","")).startswith("ERROR") else "ERROR"
    return (
        f"{header}\n"
        f"  status: {status} | latency: {lat} | attempts: {att}\n"
        f"  summary: {c.get('summary','')}\n"
    )

async def call_with_retry(model, prompt, client, options, retries=3):
    last_txt, last_lat = None, None
    for k in range(1, retries + 1):
        txt, lat = await call_ollama(model, prompt, client, options)
        if isinstance(txt, str) and not txt.startswith("ERROR"):
            return txt, lat, k
        last_txt, last_lat = txt, lat
        await asyncio.sleep(0.5 * k)
    return last_txt, last_lat, retries

async def run_eval_homo18(
    model="llama3.1:8b",
    max_samples=None,
    agents=18,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    write_text_log=True,
    options=RUN_OPTIONS,
):
    import httpx
    from tqdm import tqdm

    if not os.path.exists(DATASET_PATH):
        print(f"ERROR: Dataset not found at {DATASET_PATH}. Upload samsum.jsonl to /content/")
        return

    if not await ollama_is_up():
        print("ERROR: Ollama not reachable at http://127.0.0.1:11434")
        print("Try: !tail -n 120 /tmp/ollama.log")
        return

    dataset = []
    with open(DATASET_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(json.loads(line))

    if max_samples is not None:
        dataset = dataset[:max_samples]

    run_id = f"colab_homo18_{model.replace(':','_').replace('/','_')}_{int(time.time())}"
    run_dir = Path(RESULTS_DIR) / run_id
    run_dir.mkdir(parents=True, exist_ok=True)
    out_file = run_dir / "outputs.jsonl"
    log_file = run_dir / "run_log.txt"

    print(f"Starting Homogeneous-18 Evaluation: {run_id}")
    print(f"Results saved to: {out_file}")
    if write_text_log:
        print(f"Text log saved to: {log_file}")
    print(f"Model: {model} | agents={agents} | concurrency={concurrency} | samples={len(dataset)}")
    print("-" * 80)

    sem = asyncio.Semaphore(concurrency)

    def _log_line(s: str):
        if write_text_log:
            with open(log_file, "a", encoding="utf-8") as lf:
                lf.write(s + "\n")

    async with httpx.AsyncClient(timeout=300.0) as client:
        for idx_item, item in enumerate(tqdm(dataset, desc="Processing Samples")):
            prompt = f"Summarize this message in one short sentence:\n\n{item['message']}\n\nSummary:"

            async def _one(agent_idx: int):
                async with sem:
                    txt, lat, attempts = await call_with_retry(model, prompt, client, options, retries=3)
                    return agent_idx, txt, lat, attempts

            results = await asyncio.gather(*[_one(i) for i in range(agents)])

            candidates = []
            for (agent_idx, txt, lat, attempts) in results:
                candidates.append({
                    "model": model,
                    "agent_idx": int(agent_idx),
                    "summary": txt,
                    "latency": float(lat),
                    "attempts": int(attempts),
                })

            valid_summaries = [
                c["summary"] for c in candidates
                if isinstance(c["summary"], str) and not c["summary"].startswith("ERROR")
            ]

            if valid_summaries:
                best_idx_valid, meta = get_consensus(valid_summaries)
                swarm_summary = valid_summaries[best_idx_valid]
            else:
                meta = {}
                swarm_summary = "ERROR: All agents failed"

            do_print = print_each and ((idx_item % print_every) == 0)
            if do_print:
                sep = "=" * 90
                header = f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}"
                msg_preview = _truncate(item["message"].replace("\n", " "), 400)

                print("\n" + sep)
                print(header)
                print("MESSAGE (preview):", msg_preview)
                print("-" * 90)

                _log_line("\n" + sep)
                _log_line(header)
                _log_line("MESSAGE:")
                _log_line(item["message"])
                _log_line("-" * 90)

                for j, c in enumerate(candidates):
                    c_out = dict(c)
                    if truncate_console and isinstance(c_out["summary"], str):
                        c_out["summary"] = _truncate(c_out["summary"], truncate_console)
                    block = _format_candidate_block(c_out, idx=j)
                    print(block)
                    _log_line(_format_candidate_block(c, idx=j))

                print("-" * 90)
                print("CONSENSUS avg_similarity:", meta.get("avg_similarity"))
                if meta.get("consensus_scores") is not None:
                    print("CONSENSUS scores:", meta.get("consensus_scores"))
                print("SWARM SUMMARY:", swarm_summary)
                print(sep + "\n")

                _log_line("-" * 90)
                _log_line(f"CONSENSUS avg_similarity: {meta.get('avg_similarity')}")
                _log_line(f"CONSENSUS scores: {meta.get('consensus_scores')}")
                _log_line(f"SWARM SUMMARY: {swarm_summary}")
                _log_line(sep)

            result = {
                "id": item.get("id"),
                "model": model,
                "agents": int(agents),
                "swarm_summary": swarm_summary,
                "swarm_candidates": candidates,
                "consensus_metadata": meta,
                "factuality": check_factuality(item["message"], swarm_summary),
            }

            with open(out_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(result) + "\n")

    print(f"\n✓ Done. outputs.jsonl: {out_file}")
    if write_text_log:
        print(f"✓ Done. run_log.txt:  {log_file}")

# Run (default: llama3.1:8b, 18 agents)
await run_eval_homo18(
    model="llama3.1:8b",
    max_samples=None,
    agents=18,
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    options=RUN_OPTIONS
)


^C
Starting Homogeneous-18 Evaluation: colab_homo18_llama3.1_8b_1767416982
Results saved to: /content/slm_evaluation/results/colab_homo18_llama3.1_8b_1767416982/outputs.jsonl
Text log saved to: /content/slm_evaluation/results/colab_homo18_llama3.1_8b_1767416982/run_log.txt
Model: llama3.1:8b | agents=18 | concurrency=6 | samples=819
--------------------------------------------------------------------------------


Processing Samples:   0%|          | 1/819 [00:05<1:13:36,  5.40s/it]


SAMPLE 1/819 | ID: samsum_test_0000
MESSAGE (preview): Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda:  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 3.77s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

AGENT 02: llama3.1:8b
  status: OK | latency: 3.77s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

AGENT 03: llama3.1:8b
  status: OK | latency: 3.78s | attempts: 1
  summary: Hannah asked Amanda for Be

Processing Samples:   1%|▏         | 11/819 [00:37<41:35,  3.09s/it]


SAMPLE 11/819 | ID: samsum_test_0010
MESSAGE (preview): Wanda: Let's make a party! Gina: Why? Wanda: beacuse. I want some fun! Gina: ok, what do u need? Wanda: 1st I need too make a list Gina: noted and then? Wanda: well, could u take yours father car and go do groceries with me? Gina: don't know if he'll agree Wanda: I know, but u can ask :) Gina: I'll try but theres no promisess Wanda: I know, u r the best! Gina: When u wanna go Wanda: Friday? Gina:  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.07s | attempts: 1
  summary: Wanda wants to have a party and asks Gina for help with errands so they can plan together.

AGENT 02: llama3.1:8b
  status: OK | latency: 1.10s | attempts: 1
  summary: Wanda wants to have a party and needs Gina's help with errands before it can happen.

AGENT 03: llama3.1:8b
  status: OK | latency: 1.11s | attempts: 1
  summary: Wanda wants to have a party

...


SAMPLE 801/819 | ID: samsum_test_0800
MESSAGE (preview): Linda: Hi Dad, I want to buy flowers for mum! But I don't remember which one she likes :( Michael: Well, she likes all the flowers I believe Linda: That doesn't help! I'm on a flower market right now! Michael: Send me some pics then Linda: <file_photo>  Michael: Tulips are nice, roses too Linda:  What about carnations? Michael: No, carnations are boring :D Linda: Thanks Dad, srsly… Michael:  What  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.26s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mother and he guides her through photos of different options.

AGENT 02: llama3.1:8b
  status: OK | latency: 1.31s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mother and he guides her through the process by sending suggestions based on photos.

AGENT 03

Processing Samples:  99%|█████████▉| 811/819 [41:43<00:22,  2.76s/it]


SAMPLE 811/819 | ID: samsum_test_0810
MESSAGE (preview): Luke: Ryanair to cancel flights affecting 400,000 more passengers. So unlucky Erin! Erin: Yeeeep...it touched me as well finally 😩😩😩😩 Now they screw all my plans to go to Egypt... Luke: You’re not the only one Erin: Good to know that my feelings are shared with other 399,999 people 😩 Nathan: Relevant news story for everyone to consider: http://waterfordwhispersnews.com/2017/09/18/ryanair-cancel-ef ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01: llama3.1:8b
  status: OK | latency: 1.17s | attempts: 1
  summary: Ryanair has announced it will cancel more flights, affecting over 400,000 passengers.

AGENT 02: llama3.1:8b
  status: OK | latency: 1.17s | attempts: 1
  summary: Ryanair has announced it will cancel more flights, affecting around 400,000 passengers.

AGENT 03: llama3.1:8b
  status: OK | latency: 1.01s | attempts: 1
  summary: Ryanair has cancelled flight

Processing Samples: 100%|██████████| 819/819 [42:10<00:00,  3.09s/it]


✓ Done. outputs.jsonl: /content/slm_evaluation/results/colab_homo18_llama3.1_8b_1767416982/outputs.jsonl
✓ Done. run_log.txt:  /content/slm_evaluation/results/colab_homo18_llama3.1_8b_1767416982/run_log.txt


In [73]:
!pkill -f "ollama serve" || true
!nohup env \
  OLLAMA_MODELS="/content/" \
  OLLAMA_KEEP_ALIVE="24h" \
  OLLAMA_MAX_LOADED_MODELS="18" \
  OLLAMA_NUM_PARALLEL="16" \
  OLLAMA_MAX_QUEUE="2048" \
  ollama serve > /tmp/ollama.log 2>&1 &

import os, time, json, asyncio, math
from pathlib import Path

# Assumes these exist already:
# DATASET_PATH, RESULTS_DIR, RUN_OPTIONS
# call_ollama(model, prompt, client, options) -> (txt, latency)
# ollama_is_up() -> bool
# get_consensus(list_of_strings) -> (best_idx, meta)
# check_factuality(message, summary) -> any

def _truncate(s: str, n: int = 400):
    if s is None:
        return ""
    s = str(s)
    return s if len(s) <= n else s[:n] + " ...[truncated]"

def _format_candidate_block(c, idx=None):
    header = f"AGENT {idx+1:02d}" if idx is not None else "AGENT"
    lat = f"{c.get('latency', 0.0):.2f}s"
    att = c.get("attempts", 1)
    status = "OK" if not str(c.get("summary","")).startswith("ERROR") else "ERROR"
    corr = "CORRECT" if c.get("is_correct_prompt") else "INCORRECT"
    return (
        f"{header} [{corr}]\n"
        f"  status: {status} | latency: {lat} | attempts: {att}\n"
        f"  summary: {c.get('summary','')}\n"
    )

async def call_with_retry(model, prompt, client, options, retries=3):
    last_txt, last_lat = None, None
    for k in range(1, retries + 1):
        txt, lat = await call_ollama(model, prompt, client, options)
        if isinstance(txt, str) and not txt.startswith("ERROR"):
            return txt, lat, k
        last_txt, last_lat = txt, lat
        await asyncio.sleep(0.5 * k)
    return last_txt, last_lat, retries

def _counts_for_k(k: int):
    if k == 3:
        return 2, 1
    c = int(math.ceil(0.6 * k))
    w = k - c
    return c, w

async def run_eval_homo_mixed_3_6_12_18(
    model="llama3.1:8b",
    max_samples=None,
    sizes=(3, 6, 12, 18),
    concurrency=6,
    print_each=True,
    print_every=10,
    truncate_console=300,
    write_text_log=True,
    options=RUN_OPTIONS,
    seed=1337,
):
    import httpx
    from tqdm import tqdm
    import random

    sizes = tuple(sorted(set(int(x) for x in sizes)))
    assert sizes == (3, 6, 12, 18), f"Expected sizes (3,6,12,18), got {sizes}"

    if not os.path.exists(DATASET_PATH):
        print(f"ERROR: Dataset not found at {DATASET_PATH}. Upload samsum.jsonl to /content/")
        return

    if not await ollama_is_up():
        print("ERROR: Ollama not reachable at http://127.0.0.1:11434")
        print("Try: !tail -n 120 /tmp/ollama.log")
        return

    dataset = []
    with open(DATASET_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(json.loads(line))

    if max_samples is not None:
        dataset = dataset[:max_samples]

    # Compute max needed once (for k=18)
    max_k = max(sizes)
    max_correct, max_wrong = _counts_for_k(max_k)
    assert max_correct + max_wrong == max_k

    run_id = f"colab_homo_mixed_3_6_12_18_{model.replace(':','_').replace('/','_')}_{int(time.time())}"
    run_dir = Path(RESULTS_DIR) / run_id
    run_dir.mkdir(parents=True, exist_ok=True)
    out_file = run_dir / "outputs.jsonl"
    log_file = run_dir / "run_log.txt"

    print(f"Starting Mixed-Prompt Homogeneous Evaluation: {run_id}")
    print(f"Results saved to: {out_file}")
    if write_text_log:
        print(f"Text log saved to: {log_file}")
    print(f"Model: {model} | sizes={sizes} | per-sample calls={max_k} | concurrency={concurrency} | samples={len(dataset)}")
    print(f"Max mix for k={max_k}: correct={max_correct}, incorrect={max_wrong}")
    print("-" * 80)

    sem = asyncio.Semaphore(concurrency)

    def _log_line(s: str):
        if write_text_log:
            with open(log_file, "a", encoding="utf-8") as lf:
                lf.write(s + "\n")

    rng = random.Random(seed)

    async with httpx.AsyncClient(timeout=300.0) as client:
        for idx_item, item in enumerate(tqdm(dataset, desc="Processing Samples")):
            msg = item["message"]

            # Correct prompt (normal summarization)
            prompt_correct = (
                "Summarize this message in one short sentence.\n\n"
                f"{msg}\n\nSummary:"
            )

            # Incorrect prompt: intentionally wrong summary for robustness testing
            # This is explicit to avoid accidental “creative paraphrase” being treated as incorrect.
            prompt_incorrect = (
                "Summarize this message in one short sentence, but make the summary intentionally incorrect.\n"
                "Change key facts (who did what, what happened, or the outcome) so it contradicts the message.\n\n"
                f"{msg}\n\nIntentionally incorrect summary:"
            )

            # Build a randomized order of correct/incorrect calls for this sample
            # We will still keep them in two buckets (correct list + incorrect list) for easy slicing later.
            flags = [True] * max_correct + [False] * max_wrong
            rng.shuffle(flags)

            async def _one(agent_idx: int, is_correct: bool):
                async with sem:
                    p = prompt_correct if is_correct else prompt_incorrect
                    txt, lat, attempts = await call_with_retry(model, p, client, options, retries=3)
                    return {
                        "model": model,
                        "agent_idx": int(agent_idx),
                        "is_correct_prompt": bool(is_correct),
                        "summary": txt,
                        "latency": float(lat),
                        "attempts": int(attempts),
                    }

            # One batch per sample (18 calls), reused for all sizes
            batch = await asyncio.gather(*[_one(i, flags[i]) for i in range(max_k)])

            correct_candidates = [c for c in batch if c["is_correct_prompt"]]
            wrong_candidates = [c for c in batch if not c["is_correct_prompt"]]

            # Ensure we have enough of each type
            if len(correct_candidates) < max_correct or len(wrong_candidates) < max_wrong:
                # This should not happen, but keep it safe
                correct_candidates = (correct_candidates + wrong_candidates)[:max_correct]
                wrong_candidates = wrong_candidates[:max_wrong]

            results_by_size = {}

            for k in sizes:
                c_need, w_need = _counts_for_k(k)
                chosen = correct_candidates[:c_need] + wrong_candidates[:w_need]

                valid_summaries = [
                    c["summary"] for c in chosen
                    if isinstance(c["summary"], str) and not c["summary"].startswith("ERROR")
                ]

                if valid_summaries:
                    best_idx_valid, meta = get_consensus(valid_summaries)
                    swarm_summary = valid_summaries[best_idx_valid]
                else:
                    meta = {}
                    swarm_summary = "ERROR: All agents failed"

                results_by_size[str(k)] = {
                    "k": int(k),
                    "correct_needed": int(c_need),
                    "incorrect_needed": int(w_need),
                    "swarm_summary": swarm_summary,
                    "swarm_candidates": chosen,
                    "consensus_metadata": meta,
                    "factuality": check_factuality(msg, swarm_summary),
                }

            do_print = print_each and ((idx_item % print_every) == 0)
            if do_print:
                sep = "=" * 90
                header = f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}"
                msg_preview = _truncate(msg.replace("\n", " "), 400)

                print("\n" + sep)
                print(header)
                print("MESSAGE (preview):", msg_preview)
                print("-" * 90)

                _log_line("\n" + sep)
                _log_line(header)
                _log_line("MESSAGE:")
                _log_line(msg)
                _log_line("-" * 90)

                # Print only the k=18 candidate blocks (full set), since smaller ks are subsets
                chosen18 = results_by_size["18"]["swarm_candidates"]
                for j, c in enumerate(chosen18):
                    c_out = dict(c)
                    if truncate_console and isinstance(c_out["summary"], str):
                        c_out["summary"] = _truncate(c_out["summary"], truncate_console)
                    block = _format_candidate_block(c_out, idx=j)
                    print(block)
                    _log_line(_format_candidate_block(c, idx=j))

                print("-" * 90)
                for k in sizes:
                    r = results_by_size[str(k)]
                    print(f"k={k} | correct={r['correct_needed']} incorrect={r['incorrect_needed']} | avg_similarity={r['consensus_metadata'].get('avg_similarity')}")
                    print("SWARM SUMMARY:", r["swarm_summary"])
                    print("-" * 90)

                print(sep + "\n")

            record = {
                "id": item.get("id"),
                "model": model,
                "scenario": "mixed_prompts_incorrect_injection",
                "sizes": list(sizes),
                "max_k": int(max_k),
                "seed": int(seed),
                "results_by_size": results_by_size,
            }

            with open(out_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(record) + "\n")

    print(f"\n✓ Done. outputs.jsonl: {out_file}")
    if write_text_log:
        print(f"✓ Done. run_log.txt:  {log_file}")

# Run
await run_eval_homo_mixed_3_6_12_18(
    model="llama3.1:8b",
    max_samples=None,
    concurrency=6,
    print_each=True,
    print_every=100,
    truncate_console=300,
    options=RUN_OPTIONS,
    seed=1337,
)


^C
Starting Mixed-Prompt Homogeneous Evaluation: colab_homo_mixed_3_6_12_18_llama3.1_8b_1767419513
Results saved to: /content/slm_evaluation/results/colab_homo_mixed_3_6_12_18_llama3.1_8b_1767419513/outputs.jsonl
Text log saved to: /content/slm_evaluation/results/colab_homo_mixed_3_6_12_18_llama3.1_8b_1767419513/run_log.txt
Model: llama3.1:8b | sizes=(3, 6, 12, 18) | per-sample calls=18 | concurrency=6 | samples=819
Max mix for k=18: correct=11, incorrect=7
--------------------------------------------------------------------------------


Processing Samples:   0%|          | 1/819 [00:06<1:23:00,  6.09s/it]


SAMPLE 1/819 | ID: samsum_test_0000
MESSAGE (preview): Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda:  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01 [CORRECT]
  status: OK | latency: 3.74s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

AGENT 02 [CORRECT]
  status: OK | latency: 3.74s | attempts: 1
  summary: Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Larry instead.

AGENT 03 [CORRECT]
  status: OK | latency: 3.68s | attempts: 1
  summary: Hannah asked Amanda for Betty's pho

Processing Samples:  12%|█▏        | 101/819 [06:50<51:42,  4.32s/it]


SAMPLE 101/819 | ID: samsum_test_0100
MESSAGE (preview): Chris: Hi there! Where are you? Any chance of skyping? Rick: Hi! Our last two days in Cancun before flying to Havana. Yeah, skyping is an idea. When would it suit you? Rick: We don't have the best of connections in the room but I can get you pretty well in the lobby. Chris: What's the time in your place now? Rick: 6:45 pm Chris: It's a quarter to one in the morning here. Am still in front of the b ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01 [CORRECT]
  status: OK | latency: 1.60s | attempts: 1
  summary: Chris and Rick discuss and arrange a Skype call for the next day at 3pm Rick's time, when he is in Cuba.

AGENT 02 [CORRECT]
  status: OK | latency: 1.52s | attempts: 1
  summary: Chris and Rick discuss and agree on a Skype call for the next day at 3pm, when they are in Cuba.

AGENT 03 [CORRECT]
  status: OK | latency: 1.66s | attempts: 1
  summary: Chris and

...


SAMPLE 701/819 | ID: samsum_test_0700
MESSAGE (preview): Charlie: How are you doing. Daisy: Good, seeing a lot of beautiful places Charlie: Where are you now? Daisy: in Naples  Charlie: Pity, I hoped you may be still in Sicily. Daisy: No, we took a ferry early in the morning yesterday. Charlie: I am still thinking about the evening we spent together. Daisy: Yes, it was nice. Charlie: I would like to meet you again. Daisy: But where? Charlie: Either in t ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01 [CORRECT]
  status: OK | latency: 1.65s | attempts: 1
  summary: Charlie and Daisy plan to meet up in Florence, Italy on September 1st after Charlie decides to take a train there from Naples.

AGENT 02 [CORRECT]
  status: OK | latency: 1.64s | attempts: 1
  summary: Charlie and Daisy, who had a romantic evening together in Sicily, plan to meet again in Florence, Italy on September 1-5.

AGENT 03 [CORRECT]
  status: OK | la

Processing Samples:  98%|█████████▊| 801/819 [53:57<01:12,  4.05s/it]


SAMPLE 801/819 | ID: samsum_test_0800
MESSAGE (preview): Linda: Hi Dad, I want to buy flowers for mum! But I don't remember which one she likes :( Michael: Well, she likes all the flowers I believe Linda: That doesn't help! I'm on a flower market right now! Michael: Send me some pics then Linda: <file_photo>  Michael: Tulips are nice, roses too Linda:  What about carnations? Michael: No, carnations are boring :D Linda: Thanks Dad, srsly… Michael:  What  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01 [CORRECT]
  status: OK | latency: 1.54s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mother and he guides her through the process by suggesting specific types of flowers.

AGENT 02 [CORRECT]
  status: OK | latency: 1.53s | attempts: 1
  summary: Linda asks her dad for help choosing flowers to buy for their mother and he assists her by suggesting specific types of flowers.

AGENT 03

Processing Samples: 100%|██████████| 819/819 [55:05<00:00,  4.04s/it]


✓ Done. outputs.jsonl: /content/slm_evaluation/results/colab_homo_mixed_3_6_12_18_llama3.1_8b_1767419513/outputs.jsonl
✓ Done. run_log.txt:  /content/slm_evaluation/results/colab_homo_mixed_3_6_12_18_llama3.1_8b_1767419513/run_log.txt


In [77]:
!pkill -f "ollama serve" || true
!nohup env \
  OLLAMA_MODELS="/content/" \
  OLLAMA_KEEP_ALIVE="24h" \
  OLLAMA_MAX_LOADED_MODELS="18" \
  OLLAMA_NUM_PARALLEL="16" \
  OLLAMA_MAX_QUEUE="2048" \
  ollama serve > /tmp/ollama.log 2>&1 &
import os, time, json, asyncio
from pathlib import Path

# Assumes these already exist in your notebook:
# DATASET_PATH, RESULTS_DIR, RUN_OPTIONS
# call_ollama(model, prompt, client, options) -> (txt, latency)
# ollama_is_up() -> bool
# get_consensus(list_of_strings) -> (best_idx, meta)
# check_factuality(message, summary) -> any

async def call_with_retry(model, prompt, client, options, retries=3):
    last_txt, last_lat = None, None
    for k in range(1, retries + 1):
        txt, lat = await call_ollama(model, prompt, client, options)
        if isinstance(txt, str) and not txt.startswith("ERROR"):
            return txt, lat, k
        last_txt, last_lat = txt, lat
        await asyncio.sleep(0.5 * k)
    return last_txt, last_lat, retries

async def run_eval_k3_same_prompt(
    run_name: str,
    models: list,
    max_samples=None,
    concurrency=3,
    print_each=True,
    print_every=10,
    truncate_console=300,
    write_text_log=True,
    options=RUN_OPTIONS,
):
    import httpx
    from tqdm import tqdm

    assert len(models) == 3, f"Expected exactly 3 models, got {len(models)}"

    if not os.path.exists(DATASET_PATH):
        print(f"ERROR: Dataset not found at {DATASET_PATH}. Upload samsum.jsonl to /content/")
        return

    if not await ollama_is_up():
        print("ERROR: Ollama not reachable at http://127.0.0.1:11434")
        print("Try: !tail -n 120 /tmp/ollama.log")
        return

    dataset = []
    with open(DATASET_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(json.loads(line))

    if max_samples is not None:
        dataset = dataset[:max_samples]

    run_id = f"{run_name}_{int(time.time())}"
    run_dir = Path(RESULTS_DIR) / run_id
    run_dir.mkdir(parents=True, exist_ok=True)
    out_file = run_dir / "outputs.jsonl"
    log_file = run_dir / "run_log.txt"

    print(f"Starting: {run_id}")
    print(f"Models: {models}")
    print(f"Results saved to: {out_file}")
    if write_text_log:
        print(f"Text log saved to: {log_file}")
    print("-" * 80)

    sem = asyncio.Semaphore(concurrency)

    def _log_line(s: str):
        if write_text_log:
            with open(log_file, "a", encoding="utf-8") as lf:
                lf.write(s + "\n")

    def _truncate(s: str, n: int):
        if s is None:
            return ""
        s = str(s)
        return s if len(s) <= n else s[:n] + " ...[truncated]"

    async with httpx.AsyncClient(timeout=300.0) as client:
        for idx_item, item in enumerate(tqdm(dataset, desc="Processing Samples")):
            msg = item["message"]
            prompt = f"Summarize this message in one short sentence:\n\n{msg}\n\nSummary:"

            async def _one(i: int, m: str):
                async with sem:
                    txt, lat, attempts = await call_with_retry(m, prompt, client, options, retries=3)
                    return {
                        "model": m,
                        "agent_idx": int(i),
                        "summary": txt,
                        "latency": float(lat) if lat is not None else None,
                        "attempts": int(attempts),
                    }

            candidates = await asyncio.gather(*[_one(i, models[i]) for i in range(3)])

            valid_summaries = [
                c["summary"] for c in candidates
                if isinstance(c["summary"], str) and not c["summary"].startswith("ERROR")
            ]

            if valid_summaries:
                best_idx, meta = get_consensus(valid_summaries)
                swarm_summary = valid_summaries[best_idx]
            else:
                meta = {}
                swarm_summary = "ERROR: All agents failed"

            do_print = print_each and ((idx_item % print_every) == 0)
            if do_print:
                print("\n" + "=" * 90)
                print(f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}")
                print("MESSAGE (preview):", _truncate(msg.replace("\n", " "), 400))
                print("-" * 90)
                for j, c in enumerate(candidates):
                    s = c["summary"]
                    if truncate_console and isinstance(s, str):
                        s = _truncate(s, truncate_console)
                    print(f"AGENT {j+1:02d} | model={c['model']} | latency={float(c.get('latency') or 0.0):.2f}s | attempts={c['attempts']}")
                    print(" ", s)
                print("-" * 90)
                print("CONSENSUS avg_similarity:", meta.get("avg_similarity"))
                print("SWARM SUMMARY:", swarm_summary)
                print("=" * 90 + "\n")

                _log_line("=" * 90)
                _log_line(f"SAMPLE {idx_item+1}/{len(dataset)} | ID: {item.get('id','NA')}")
                _log_line("MESSAGE:")
                _log_line(msg)
                _log_line("-" * 90)
                for j, c in enumerate(candidates):
                    _log_line(f"AGENT {j+1:02d} | model={c['model']} | latency={c.get('latency')} | attempts={c['attempts']}")
                    _log_line(c.get("summary", ""))
                _log_line("-" * 90)
                _log_line(f"CONSENSUS avg_similarity: {meta.get('avg_similarity')}")
                _log_line(f"SWARM SUMMARY: {swarm_summary}")
                _log_line("=" * 90)

            record = {
                "id": item.get("id"),
                "run_name": run_name,
                "k": 3,
                "models": models,
                "swarm_summary": swarm_summary,
                "swarm_candidates": candidates,
                "consensus_metadata": meta,
                "factuality": check_factuality(msg, swarm_summary),
            }

            with open(out_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(record) + "\n")

    print(f"\n✓ Done. outputs.jsonl: {out_file}")
    if write_text_log:
        print(f"✓ Done. run_log.txt:  {log_file}")

# ------------------------
# HOMO-3: same model x3
# ------------------------
async def run_homo3(model="llama3.1:8b", **kwargs):
    await run_eval_k3_same_prompt(
        run_name=f"colab_homo3_{model.replace(':','_').replace('/','_')}",
        models=[model, model, model],
        **kwargs
    )

# ------------------------
# HETRO-3: 3 different models
# ------------------------
async def run_hetero3(models=("llama3.1:8b", "qwen2.5:7b-instruct", "mistral:7b-instruct"), **kwargs):
    await run_eval_k3_same_prompt(
        run_name="colab_hetero3",
        models=list(models),
        **kwargs
    )

# Example:
await run_homo3(model="llama3.1:8b", max_samples=None, concurrency=10, print_every=100, options=RUN_OPTIONS)
# await run_hetero3(models=("llama3.1:8b","qwen2.5:7b-instruct","gemma:7b-instruct"), max_samples=50, concurrency=3, options=RUN_OPTIONS)


^C
Starting: colab_homo3_llama3.1_8b_1767423259
Models: ['llama3.1:8b', 'llama3.1:8b', 'llama3.1:8b']
Results saved to: /content/slm_evaluation/results/colab_homo3_llama3.1_8b_1767423259/outputs.jsonl
Text log saved to: /content/slm_evaluation/results/colab_homo3_llama3.1_8b_1767423259/run_log.txt
--------------------------------------------------------------------------------


Processing Samples:   0%|          | 1/819 [00:03<47:33,  3.49s/it]


SAMPLE 1/819 | ID: samsum_test_0000
MESSAGE (preview): Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda:  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01 | model=llama3.1:8b | latency=3.43s | attempts=1
  Hannah asked Amanda for Betty's phone number, but Amanda suggested she ask Larry instead.
AGENT 02 | model=llama3.1:8b | latency=3.43s | attempts=1
  Hannah asked Amanda for Betty's phone number, but Amanda suggested she ask Larry instead.
AGENT 03 | model=llama3.1:8b | latency=3.45s | attempts=1
  Hannah asked Amanda for Betty's phone number, but Amanda suggested asking Lar

Processing Samples:  12%|█▏        | 101/819 [01:24<09:04,  1.32it/s]


SAMPLE 101/819 | ID: samsum_test_0100
MESSAGE (preview): Chris: Hi there! Where are you? Any chance of skyping? Rick: Hi! Our last two days in Cancun before flying to Havana. Yeah, skyping is an idea. When would it suit you? Rick: We don't have the best of connections in the room but I can get you pretty well in the lobby. Chris: What's the time in your place now? Rick: 6:45 pm Chris: It's a quarter to one in the morning here. Am still in front of the b ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01 | model=llama3.1:8b | latency=0.87s | attempts=1
  Chris and Rick discuss scheduling a Skype call for the next day at 3pm, when they are in Cuba and Chris is back home.
AGENT 02 | model=llama3.1:8b | latency=0.81s | attempts=1
  Chris and Rick discuss scheduling a Skype call for the next day at 3pm, when they are in Cuba.
AGENT 03 | model=llama3.1:8b | latency=0.82s | attempts=1
  Chris and Rick discuss and arrange a Skype

...


SAMPLE 701/819 | ID: samsum_test_0700
MESSAGE (preview): Charlie: How are you doing. Daisy: Good, seeing a lot of beautiful places Charlie: Where are you now? Daisy: in Naples  Charlie: Pity, I hoped you may be still in Sicily. Daisy: No, we took a ferry early in the morning yesterday. Charlie: I am still thinking about the evening we spent together. Daisy: Yes, it was nice. Charlie: I would like to meet you again. Daisy: But where? Charlie: Either in t ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01 | model=llama3.1:8b | latency=0.71s | attempts=1
  Charlie and Daisy make plans to meet up again in Florence, Italy on September 1st.
AGENT 02 | model=llama3.1:8b | latency=0.74s | attempts=1
  Charlie and Daisy make plans to meet up again in Florence, Italy on September 1st.
AGENT 03 | model=llama3.1:8b | latency=0.84s | attempts=1
  Charlie and Daisy, who had a romantic evening together in Sicily, plan to meet again in F

Processing Samples:  98%|█████████▊| 801/819 [10:43<00:14,  1.24it/s]


SAMPLE 801/819 | ID: samsum_test_0800
MESSAGE (preview): Linda: Hi Dad, I want to buy flowers for mum! But I don't remember which one she likes :( Michael: Well, she likes all the flowers I believe Linda: That doesn't help! I'm on a flower market right now! Michael: Send me some pics then Linda: <file_photo>  Michael: Tulips are nice, roses too Linda:  What about carnations? Michael: No, carnations are boring :D Linda: Thanks Dad, srsly… Michael:  What  ...[truncated]
------------------------------------------------------------------------------------------
AGENT 01 | model=llama3.1:8b | latency=0.82s | attempts=1
  Linda asks her dad for help choosing flowers to buy for their mother and he guides her through the process by suggesting specific types of flowers.
AGENT 02 | model=llama3.1:8b | latency=0.72s | attempts=1
  Linda asks her dad for help choosing flowers to buy for their mother and he guides her through the options.
AGENT 03 | model=llama3.1:8b | latency=0.75s | attempts=1
 

Processing Samples: 100%|██████████| 819/819 [10:56<00:00,  1.25it/s]


✓ Done. outputs.jsonl: /content/slm_evaluation/results/colab_homo3_llama3.1_8b_1767423259/outputs.jsonl
✓ Done. run_log.txt:  /content/slm_evaluation/results/colab_homo3_llama3.1_8b_1767423259/run_log.txt


In [78]:
!cp -r /content/slm_evaluation /content/drive/MyDrive/slm_evaluation_new